In [147]:
import os
import time
from IPython.display import clear_output, display, HTML
def show_figs(file_name="graph.png", directory=""):
    s = """<figure>\n\t<img src="{0}?{1}" alt="The figure" >\n</figure>""".format(os.path.join(directory, file_name),time.time())
    display(HTML(s))

In [152]:
import pandas


def split_or_none(v):
    if pandas.isnull(v): return []
    return [x.strip() for x in v.split(',')]

class Recipe(object):
    def __init__(self, name='Unknown', page=-1, tool=None, replaceable_dependencies=None, strict_dependencies=None, serve_with=None):
        self.name=name
        self.tools=split_or_none(tool)
        self.page=page
        self.replaceable_dependencies = split_or_none(replaceable_dependencies)
        self.strict_dependencies = split_or_none(strict_dependencies)
        self.serve_with = split_or_none(serve_with)
    
    def __hash__(self):
        return self.page
        
    def __repr__(self):
        return "[{}] {}".format(self.page, self.name)


data = pandas.read_csv('tacos.csv')
keys = [x.lower().replace(' ', '_') for x in data]
recipes = {}
for _, v in data.iterrows():
    recipe = Recipe(**dict(zip(keys, v)))
    recipes[recipe.page] = recipe
    



In [211]:
def make_graph(searches = []):
    
    G = pygraphviz.AGraph(directed=True, rankdir="LR", style='filled')
    G.node_attr['style']='filled'
    allowed_tools = ['oven', 'grill', 'microwave']
    special_tool = 'assemble'
    for recipe in recipes.values():
        color = 'red' if any([x not in allowed_tools for x in recipe.tools]) else 'black'
        color = 'green' if special_tool in recipe.tools else color
        fillcolor = 'yellow' if any(search in recipe.name.lower() for search in searches) else 'white'
        n = G.add_node(recipe, color=color, fillcolor=fillcolor)
    for recipe in recipes.values():
        for other in (recipes[x] for x in recipe.strict_dependencies):
            G.add_edge(recipe, other)
        for other in (recipes[x] for x in recipe.replaceable_dependencies):
            G.add_edge(recipe, other, color='blue')
        for other in (recipes[x] for x in recipe.serve_with):
            G.add_edge(recipe, other, style='dashed')
    if searches:
        G.add_node("Searching for {}".format(", ".join(searches)), shape='rectangle')
    # G.add_edges_from([(1,2),(1,3),(2,4),(3,6),(4,5),(4,6),(5,7),(5,8)])
    # Optional prog=[‘neato’|’dot’|’twopi’|’circo’|’fdp’|’nop’] will use specified graphviz layout method.
    G.layout(prog='dot', args="-Goverlap=false")
    G.draw('graph.png')
    show_figs('graph.png')
    
make_graph(['bean', 'potato', 'tofu'])